In [1]:
import pymysql
import matplotlib.pyplot as plt
import pandas as pd
import os

In [2]:
if not os.path.exists('./submissions.csv'):
    conn = pymysql.connect('127.0.0.1', 'root', 'password', 'anubis')
    with conn.cursor() as cursor:
        cursor.execute('select * from submission;')
        raw_submissions = cursor.fetchall()
    
        submissions = pd.DataFrame(
            data=raw_submissions, 
            columns=['id', 'owner_id', 'assignment_id', 'assignment_repo_id', 'created', 
                     'last_updated', 'commit', 'processed', 'state', 'errors', 'token'])

        submissions = submissions[['id', 'owner_id', 'assignment_id', 'processed', 'created']]
        submissions['created'] = submissions['created'].apply(lambda date: pd.to_datetime(date).round('H'))
        submissions.to_csv('submissions.csv')
    conn.close()

submissions = pd.read_csv('./submissions.csv')
submissions.head(10)

,Unnamed: 0,id,owner_id,assignment_id,processed,created
0,0,1,166.0,1,1,2020-09-10 14:00:00
1,1,2,264.0,1,1,2020-09-15 15:00:00
2,2,3,166.0,1,1,2020-09-15 16:00:00
3,3,4,197.0,1,1,2020-09-15 16:00:00
4,4,5,197.0,1,1,2020-09-15 16:00:00
5,5,6,197.0,1,1,2020-09-15 16:00:00
6,6,7,197.0,1,1,2020-09-15 17:00:00
7,7,8,197.0,1,1,2020-09-15 17:00:00
8,8,9,245.0,1,1,2020-09-15 18:00:00
9,9,10,178.0,1,1,2020-09-15 19:00:00


In [3]:
if not os.path.exists('./theia_sessions.csv'):
    conn = pymysql.connect('127.0.0.1', 'root', 'password', 'anubis')
    with conn.cursor() as cursor:
        cursor.execute('select * from theia_session;')
        raw_theia_sessions = cursor.fetchall()
    
        theia_sessions = pd.DataFrame(
            data=raw_theia_sessions, 
            columns=['id', 'owner_id', 'assignment_id', 'repo_id', 'active', 'state', 'cluster_address', 
                     'created', 'ended', 'last_heartbeat', 'last_proxy', 'last_updated'])

        theia_sessions = theia_sessions[['id', 'owner_id', 'assignment_id', 'created', 'ended']]
        theia_sessions['created'] = theia_sessions['created'].apply(lambda date: pd.to_datetime(date).round('min'))
        theia_sessions['ended'] = theia_sessions['ended'].apply(lambda date: pd.to_datetime(date).round('min'))
        theia_sessions['duration'] = theia_sessions[['ended', 'created']].apply(lambda row: (row[0] - row[1]).seconds / 60, axis=1)
        theia_sessions = theia_sessions[np.abs(theia_sessions.duration-theia_sessions.duration.mean())<=(3*theia_sessions.duration.std())] # Drop outliers based on duration
        theia_sessions.to_csv('./theia_sessions.csv')
    conn.close()

theia_sessions = pd.read_csv('./theia_sessions.csv')
theia_sessions.head(10)

,Unnamed: 0,id,owner_id,assignment_id,created,ended,duration
0,0,1,166,5,2020-12-12 23:23:00,2020-12-12 23:28:00,5.0
1,1,2,166,5,2020-12-12 23:29:00,2020-12-12 23:29:00,0.0
2,2,3,166,5,2020-12-12 23:30:00,2020-12-12 23:47:00,17.0
3,3,4,166,5,2020-12-12 23:47:00,2020-12-12 23:49:00,2.0
4,4,5,166,5,2020-12-12 23:50:00,2020-12-12 23:54:00,4.0
5,5,6,166,5,2020-12-12 23:54:00,2020-12-12 23:55:00,1.0
6,6,7,166,5,2020-12-13 00:02:00,2020-12-13 06:15:00,373.0
7,7,8,166,5,2020-12-13 16:57:00,2020-12-13 23:00:00,363.0
8,8,9,240,5,2020-12-13 23:57:00,2020-12-14 06:00:00,363.0
9,9,10,236,5,2020-12-14 19:09:00,2020-12-14 19:11:00,2.0
